In [1]:
import numpy as np

In [12]:
# Comet ML for experiment logging
from comet_ml import Experiment

In [14]:
# Import packages
import tensorflow as tf

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [20]:
# Set random seed
tf.random.set_random_seed(1994)
np.random.seed(1994)

<_io.TextIOWrapper name='ratings.dat' mode='r' encoding='UTF-8'>

In [15]:
def read_rating(num_users, num_items, num_total_ratings, a, b, train_ratio):
    """
    Function to read in the ratings data
    :param path: path to ratings data
    :param num_users: number of users
    :param num_items: number of items
    :param num_total_ratings: number of total ratings
    :param a: positive rating (1)
    :param b: negative rating (0)
    :param train_ratio: ratio that splits train and test sets
    """
    fp = open("ratings.dat")

    user_train_set = set()
    user_test_set = set()
    item_train_set = set()
    item_test_set = set()

    R = np.zeros((num_users, num_items))
    mask_R = np.zeros((num_users, num_items))
    C = np.ones((num_users, num_items)) * b

    train_R = np.zeros((num_users, num_items))
    test_R = np.zeros((num_users, num_items))

    train_mask_R = np.zeros((num_users, num_items))
    test_mask_R = np.zeros((num_users, num_items))

    random_perm_idx = np.random.permutation(num_total_ratings)
    train_idx = random_perm_idx[0:int(num_total_ratings * train_ratio)]
    test_idx = random_perm_idx[int(num_total_ratings * train_ratio):]

    num_train_ratings = len(train_idx)
    num_test_ratings = len(test_idx)

    lines = fp.readlines()
    for line in lines:
        user, item, rating, _ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        R[user_idx, item_idx] = int(rating)
        mask_R[user_idx, item_idx] = 1
        C[user_idx, item_idx] = a

    # Training set
    for itr in train_idx:
        line = lines[itr]
        user, item, rating, _ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        train_R[user_idx, item_idx] = int(rating)
        train_mask_R[user_idx, item_idx] = 1

        user_train_set.add(user_idx)
        item_train_set.add(item_idx)

    # Test set
    for itr in test_idx:
        line = lines[itr]
        user, item, rating, _ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        test_R[user_idx, item_idx] = int(rating)
        test_mask_R[user_idx, item_idx] = 1

        user_test_set.add(user_idx)
        item_test_set.add(item_idx)

    return R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings, \
           user_train_set, item_train_set, user_test_set, item_test_set

In [16]:
data_name = 'ml-1m'
num_users = 6040
num_items = 3952
num_total_ratings = 1000209
# Data is split into random 75% - 25% train-test sets
train_ratio = 0.75

In [17]:
R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings, \
user_train_set, item_train_set, user_test_set, item_test_set = read_rating(num_users, num_items, num_total_ratings, 1, 0, train_ratio)

In [21]:
# Get hyper-parameters
hyper_params = {
    "hidden_neuron": 500,
    "lambda_value": 1,
    "epochs": 500,
    "batch_size": 512,
    "optimizer": "Adam",
    "learning_rate": 0.001,
    "random_seed": 1994
}

In [27]:
# Import packages
import tensorflow as tf
import time
import numpy as np
import os
import math

In [28]:
class AutoRec:
    """
    Function to define the AutoRec model class
    """
    def __init__(self, sess, args,
                 num_users, num_items,
                 R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings,
                 user_train_set, item_train_set, user_test_set, item_test_set,
                 result_path):

        self.sess = sess
        self.args = args

        self.num_users = num_users
        self.num_items = num_items

        self.R = R
        self.mask_R = mask_R
        self.C = C
        self.train_R = train_R
        self.train_mask_R = train_mask_R
        self.test_R = test_R
        self.test_mask_R = test_mask_R
        self.num_train_ratings = num_train_ratings
        self.num_test_ratings = num_test_ratings

        self.user_train_set = user_train_set
        self.item_train_set = item_train_set
        self.user_test_set = user_test_set
        self.item_test_set = item_test_set

        self.hidden_neuron = args['hidden_neuron']
        self.train_epoch = args['train_epoch']
        self.batch_size = args['batch_size']
        self.num_batch = int(math.ceil(self.num_users / float(self.batch_size)))

        self.base_lr = args['base_lr']
        self.optimizer_method = args['optimizer_method']
        self.display_step = args['display_step']
        self.random_seed = args['random_seed']

        self.global_step = tf.Variable(0, trainable=False)
        self.decay_epoch_step = args['decay_epoch_step']
        self.decay_step = self.decay_epoch_step * self.num_batch
        self.lr = tf.compat.v1.train.exponential_decay(self.base_lr, self.global_step,
                                                       self.decay_step, 0.96, staircase=True)
        self.lambda_value = args['lambda_value']

        self.train_cost_list = []
        self.test_cost_list = []
        self.test_rmse_list = []

        self.result_path = result_path
        self.grad_clip = args['grad_clip']

    def run(self, experiment):
        """
        Function to run AutoRec
        :param experiment: CometML Experiment function
        """
        # Build AutoRec
        self.prepare_model()
        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)

        # Train and evaluate AutoRec for all epochs
        for epoch_itr in range(self.train_epoch):
            experiment.set_step(epoch_itr)
            self.train_model(epoch_itr,experiment)
            self.test_model(epoch_itr, experiment)

        # Log results
        self.make_records()

    def prepare_model(self):
        """
        Function to build AutoRec
        """
        self.input_R = tf.compat.v1.placeholder(dtype=tf.float32,
                                                shape=[None, self.num_items],
                                                name="input_R")
        self.input_mask_R = tf.compat.v1.placeholder(dtype=tf.float32,
                                                     shape=[None, self.num_items],
                                                     name="input_mask_R")

        V = tf.compat.v1.get_variable(name="V", initializer=tf.compat.v1.truncated_normal(
            shape=[self.num_items, self.hidden_neuron],
            mean=0, stddev=0.03), dtype=tf.float32)
        W = tf.compat.v1.get_variable(name="W", initializer=tf.compat.v1.truncated_normal(
            shape=[self.hidden_neuron, self.num_items],
            mean=0, stddev=0.03), dtype=tf.float32)
        mu = tf.compat.v1.get_variable(name="mu", initializer=tf.zeros(shape=self.hidden_neuron), dtype=tf.float32)
        b = tf.compat.v1.get_variable(name="b", initializer=tf.zeros(shape=self.num_items), dtype=tf.float32)

        pre_Encoder = tf.matmul(self.input_R, V) + mu
        self.Encoder = tf.nn.sigmoid(pre_Encoder)
        pre_Decoder = tf.matmul(self.Encoder, W) + b
        self.Decoder = tf.identity(pre_Decoder)

        pre_rec_cost = tf.multiply((self.input_R - self.Decoder), self.input_mask_R)
        rec_cost = tf.square(self.l2_norm(pre_rec_cost))
        pre_reg_cost = tf.square(self.l2_norm(W)) + tf.square(self.l2_norm(V))
        reg_cost = self.lambda_value * 0.5 * pre_reg_cost

        self.cost = rec_cost + reg_cost

        if self.optimizer_method == "Adam":
            optimizer = tf.compat.v1.train.AdamOptimizer(self.lr)
        elif self.optimizer_method == "RMSProp":
            optimizer = tf.compat.v1.train.RMSPropOptimizer(self.lr)
        else:
            raise ValueError("Optimizer Key ERROR")

        if self.grad_clip:
            gvs = optimizer.compute_gradients(self.cost)
            capped_gvs = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gvs]
            self.optimizer = optimizer.apply_gradients(capped_gvs, global_step=self.global_step)
        else:
            self.optimizer = optimizer.minimize(self.cost, global_step=self.global_step)

    def train_model(self, itr, experiment):
        """
        Function to train AutoRec
        :param itr: Current iteration
        :param experiment: CometML experiment
        """
        start_time = time.time()
        random_perm_doc_idx = np.random.permutation(self.num_users)

        batch_cost = 0
        for i in range(self.num_batch):
            if i == self.num_batch - 1:
                batch_set_idx = random_perm_doc_idx[i * self.batch_size:]
            elif i < self.num_batch - 1:
                batch_set_idx = random_perm_doc_idx[i * self.batch_size: (i + 1) * self.batch_size]

            _, Cost = self.sess.run(
                [self.optimizer, self.cost],
                feed_dict={self.input_R: self.train_R[batch_set_idx, :],
                           self.input_mask_R: self.train_mask_R[batch_set_idx, :]})

            batch_cost = batch_cost + Cost
        self.train_cost_list.append(batch_cost)

        if (itr + 1) % self.display_step == 0:

            print("Training //", "Epoch %d //" % (itr), " Total cost = {:.2f}".format(batch_cost),
                  "Elapsed time : %d sec" % (time.time() - start_time))

        experiment.log_metric("loss", batch_cost, step=itr)

    def test_model(self, itr, experiment):
        """
        Function to evaluate AutoRec
        :param itr: Current iteration
        :param experiment: CometML experiment
        """
        start_time = time.time()
        Cost, Decoder = self.sess.run(
            [self.cost, self.Decoder],
            feed_dict={self.input_R: self.test_R,
                       self.input_mask_R: self.test_mask_R})

        self.test_cost_list.append(Cost)

        if (itr + 1) % self.display_step == 0:
            Estimated_R = Decoder.clip(min=1, max=5)
            unseen_user_test_list = list(self.user_test_set - self.user_train_set)
            unseen_item_test_list = list(self.item_test_set - self.item_train_set)

            for user in unseen_user_test_list:
                for item in unseen_item_test_list:
                    if self.test_mask_R[user, item] == 1:  # exist in test set
                        Estimated_R[user, item] = 3

            pre_numerator = np.multiply((Estimated_R - self.test_R), self.test_mask_R)
            numerator = np.sum(np.square(pre_numerator))
            denominator = self.num_test_ratings
            RMSE = np.sqrt(numerator / float(denominator))

            self.test_rmse_list.append(RMSE)

            print("Testing //", "Epoch %d //" % (itr), " Total cost = {:.2f}".format(Cost),
                  " RMSE = {:.5f}".format(RMSE),
                  "Elapsed time : %d sec" % (time.time() - start_time))
            print("=" * 100)

        experiment.log_metric("RMSE", RMSE, step=itr)

    def make_records(self):
        """
        Function to log results
        """
        if not os.path.exists(self.result_path):
            os.makedirs(self.result_path)

        basic_info = self.result_path + "basic_info.txt"
        train_record = self.result_path + "train_record.txt"
        test_record = self.result_path + "test_record.txt"

        with open(train_record, 'w') as f:
            f.write(str("Cost:"))
            f.write('\t')
            for itr in range(len(self.train_cost_list)):
                f.write(str(self.train_cost_list[itr]))
                f.write('\t')
            f.write('\n')

        with open(test_record, 'w') as g:
            g.write(str("Cost:"))
            g.write('\t')
            for itr in range(len(self.test_cost_list)):
                g.write(str(self.test_cost_list[itr]))
                g.write('\t')
            g.write('\n')

            g.write(str("RMSE:"))
            for itr in range(len(self.test_rmse_list)):
                g.write(str(self.test_rmse_list[itr]))
                g.write('\t')
            g.write('\n')

        #with open(basic_info, 'w') as h:
        #    h.write(str(self.args))

    def l2_norm(self, tensor):
        """
        Function to apply L2 normalization
        :param tensor: TensorFlow tensor
        """
        return tf.sqrt(tf.reduce_sum(tf.square(tensor)))

In [29]:
# Initialize TensorFlow Config
config = tf.compat.v1.ConfigProto()

In [30]:
experiment = Experiment(api_key="vV3GGUE8tetqNJuD2kGxVi1M9", project_name="autoencoders-movielens1M")
experiment.log_parameters(hyper_params)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/benichou/autoencoders-movielens1m/dd3f8cc2fb8e496691ed5397bf15ddf8
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 512
COMET INFO:     epochs        : 500
COMET INFO:     hidden_neuron : 500
COMET INFO:     lambda_value  : 1
COMET INFO:     learning_rate : 0.001
COMET INFO:     optimizer     : Adam
COMET INFO:     random_seed   : 1994
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (58 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     os packages              : 1
COMET INFO: ---------------------------
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/benic

In [31]:
args = {'hidden_neuron': 500, 'lambda_value':1, 'train_epoch': 500, 'batch_size': 512, 'optimizer_method': 'Adam', 'grad_clip': False, 'base_lr': 1e-3, 'decay_epoch_step': 50, 'random_seed':1994, 'display_step':1}

In [32]:
result_path = 'results/'

In [33]:
with tf.compat.v1.Session(config=config) as sess:
    # Define the AutoRec class from `AutoRec.py`
    AutoRec = AutoRec(sess, args,
                      num_users, num_items,
                      R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings,
                      user_train_set, item_train_set, user_test_set, item_test_set,
                      result_path)
    # Run the AutoRec model
    AutoRec.run(experiment)

COMET ERROR: Failed to extract parameters from Optimizer.init()


Training // Epoch 0 //  Total cost = 4314848.73 Elapsed time : 1 sec
Testing // Epoch 0 //  Total cost = 376518.59  RMSE = 1.21647 Elapsed time : 0 sec
Training // Epoch 1 //  Total cost = 1342732.23 Elapsed time : 1 sec
Testing // Epoch 1 //  Total cost = 336349.03  RMSE = 1.15472 Elapsed time : 0 sec
Training // Epoch 2 //  Total cost = 876320.30 Elapsed time : 1 sec
Testing // Epoch 2 //  Total cost = 330854.69  RMSE = 1.14754 Elapsed time : 0 sec
Training // Epoch 3 //  Total cost = 798203.29 Elapsed time : 1 sec
Testing // Epoch 3 //  Total cost = 285656.53  RMSE = 1.06594 Elapsed time : 0 sec
Training // Epoch 4 //  Total cost = 765945.64 Elapsed time : 1 sec
Testing // Epoch 4 //  Total cost = 291632.84  RMSE = 1.07712 Elapsed time : 0 sec
Training // Epoch 5 //  Total cost = 751194.51 Elapsed time : 1 sec
Testing // Epoch 5 //  Total cost = 280178.09  RMSE = 1.05561 Elapsed time : 0 sec
Training // Epoch 6 //  Total cost = 744174.01 Elapsed time : 1 sec
Testing // Epoch 6 //  T

Training // Epoch 33 //  Total cost = 557268.95 Elapsed time : 1 sec
Testing // Epoch 33 //  Total cost = 251270.86  RMSE = 0.99905 Elapsed time : 0 sec
Training // Epoch 34 //  Total cost = 551156.94 Elapsed time : 1 sec
Testing // Epoch 34 //  Total cost = 250928.77  RMSE = 0.99832 Elapsed time : 0 sec
Training // Epoch 35 //  Total cost = 545531.61 Elapsed time : 1 sec
Testing // Epoch 35 //  Total cost = 251442.22  RMSE = 0.99932 Elapsed time : 0 sec
Training // Epoch 36 //  Total cost = 540128.72 Elapsed time : 1 sec
Testing // Epoch 36 //  Total cost = 250830.91  RMSE = 0.99805 Elapsed time : 0 sec
Training // Epoch 37 //  Total cost = 534159.66 Elapsed time : 1 sec
Testing // Epoch 37 //  Total cost = 250505.41  RMSE = 0.99736 Elapsed time : 0 sec
Training // Epoch 38 //  Total cost = 528032.06 Elapsed time : 1 sec
Testing // Epoch 38 //  Total cost = 250464.06  RMSE = 0.99723 Elapsed time : 0 sec
Training // Epoch 39 //  Total cost = 522640.91 Elapsed time : 1 sec
Testing // Ep

Testing // Epoch 65 //  Total cost = 240836.64  RMSE = 0.97611 Elapsed time : 0 sec
Training // Epoch 66 //  Total cost = 367741.52 Elapsed time : 1 sec
Testing // Epoch 66 //  Total cost = 240237.12  RMSE = 0.97475 Elapsed time : 0 sec
Training // Epoch 67 //  Total cost = 361719.47 Elapsed time : 1 sec
Testing // Epoch 67 //  Total cost = 239872.56  RMSE = 0.97398 Elapsed time : 0 sec
Training // Epoch 68 //  Total cost = 356007.02 Elapsed time : 1 sec
Testing // Epoch 68 //  Total cost = 239656.98  RMSE = 0.97345 Elapsed time : 0 sec
Training // Epoch 69 //  Total cost = 350258.82 Elapsed time : 1 sec
Testing // Epoch 69 //  Total cost = 238956.53  RMSE = 0.97191 Elapsed time : 0 sec
Training // Epoch 70 //  Total cost = 344766.15 Elapsed time : 1 sec
Testing // Epoch 70 //  Total cost = 238560.94  RMSE = 0.97104 Elapsed time : 0 sec
Training // Epoch 71 //  Total cost = 338976.28 Elapsed time : 1 sec
Testing // Epoch 71 //  Total cost = 238660.00  RMSE = 0.97117 Elapsed time : 0 se

Training // Epoch 98 //  Total cost = 209032.79 Elapsed time : 1 sec
Testing // Epoch 98 //  Total cost = 231727.02  RMSE = 0.95446 Elapsed time : 0 sec
Training // Epoch 99 //  Total cost = 205435.19 Elapsed time : 1 sec
Testing // Epoch 99 //  Total cost = 231571.97  RMSE = 0.95406 Elapsed time : 0 sec
Training // Epoch 100 //  Total cost = 201867.92 Elapsed time : 1 sec
Testing // Epoch 100 //  Total cost = 231547.92  RMSE = 0.95394 Elapsed time : 0 sec
Training // Epoch 101 //  Total cost = 198445.91 Elapsed time : 1 sec
Testing // Epoch 101 //  Total cost = 231553.20  RMSE = 0.95389 Elapsed time : 0 sec
Training // Epoch 102 //  Total cost = 195186.21 Elapsed time : 1 sec
Testing // Epoch 102 //  Total cost = 231034.61  RMSE = 0.95268 Elapsed time : 0 sec
Training // Epoch 103 //  Total cost = 192208.34 Elapsed time : 1 sec
Testing // Epoch 103 //  Total cost = 231008.39  RMSE = 0.95256 Elapsed time : 0 sec
Training // Epoch 104 //  Total cost = 189031.90 Elapsed time : 1 sec
Test

...

Training // Epoch 358 //  Total cost = 58725.19 Elapsed time : 1 sec
Testing // Epoch 358 //  Total cost = 217074.00  RMSE = 0.91899 Elapsed time : 0 sec
Training // Epoch 359 //  Total cost = 58296.04 Elapsed time : 1 sec
Testing // Epoch 359 //  Total cost = 216766.33  RMSE = 0.91822 Elapsed time : 0 sec
Training // Epoch 360 //  Total cost = 57860.85 Elapsed time : 1 sec
Testing // Epoch 360 //  Total cost = 216466.59  RMSE = 0.91764 Elapsed time : 0 sec
Training // Epoch 361 //  Total cost = 57608.35 Elapsed time : 1 sec
Testing // Epoch 361 //  Total cost = 216728.56  RMSE = 0.91824 Elapsed time : 0 sec
Training // Epoch 362 //  Total cost = 57439.81 Elapsed time : 1 sec
Testing // Epoch 362 //  Total cost = 217227.91  RMSE = 0.91957 Elapsed time : 0 sec
Training // Epoch 363 //  Total cost = 57320.44 Elapsed time : 1 sec
Testing // Epoch 363 //  Total cost = 217197.50  RMSE = 0.91955 Elapsed time : 0 sec
Training // Epoch 364 //  Total cost = 57386.95 Elapsed time : 1 sec
Testing

Testing // Epoch 390 //  Total cost = 215899.84  RMSE = 0.91686 Elapsed time : 0 sec
Training // Epoch 391 //  Total cost = 55819.86 Elapsed time : 1 sec
Testing // Epoch 391 //  Total cost = 215874.27  RMSE = 0.91677 Elapsed time : 0 sec
Training // Epoch 392 //  Total cost = 55782.62 Elapsed time : 1 sec
Testing // Epoch 392 //  Total cost = 215502.11  RMSE = 0.91585 Elapsed time : 0 sec
Training // Epoch 393 //  Total cost = 55708.62 Elapsed time : 1 sec
Testing // Epoch 393 //  Total cost = 215919.20  RMSE = 0.91694 Elapsed time : 0 sec
Training // Epoch 394 //  Total cost = 55918.58 Elapsed time : 1 sec
Testing // Epoch 394 //  Total cost = 216027.50  RMSE = 0.91722 Elapsed time : 0 sec
Training // Epoch 395 //  Total cost = 56345.07 Elapsed time : 1 sec
Testing // Epoch 395 //  Total cost = 214808.59  RMSE = 0.91398 Elapsed time : 0 sec
Training // Epoch 396 //  Total cost = 56674.65 Elapsed time : 1 sec
Testing // Epoch 396 //  Total cost = 215249.00  RMSE = 0.91536 Elapsed time

Training // Epoch 423 //  Total cost = 55779.67 Elapsed time : 1 sec
Testing // Epoch 423 //  Total cost = 214531.83  RMSE = 0.91391 Elapsed time : 0 sec
Training // Epoch 424 //  Total cost = 55911.41 Elapsed time : 1 sec
Testing // Epoch 424 //  Total cost = 213777.64  RMSE = 0.91210 Elapsed time : 0 sec
Training // Epoch 425 //  Total cost = 56405.49 Elapsed time : 1 sec
Testing // Epoch 425 //  Total cost = 215411.73  RMSE = 0.91622 Elapsed time : 0 sec
Training // Epoch 426 //  Total cost = 56730.69 Elapsed time : 1 sec
Testing // Epoch 426 //  Total cost = 213654.44  RMSE = 0.91172 Elapsed time : 0 sec
Training // Epoch 427 //  Total cost = 56250.44 Elapsed time : 1 sec
Testing // Epoch 427 //  Total cost = 214710.61  RMSE = 0.91445 Elapsed time : 0 sec
Training // Epoch 428 //  Total cost = 56356.78 Elapsed time : 1 sec
Testing // Epoch 428 //  Total cost = 214142.61  RMSE = 0.91316 Elapsed time : 0 sec
Training // Epoch 429 //  Total cost = 56109.07 Elapsed time : 1 sec
Testing

Testing // Epoch 455 //  Total cost = 212909.36  RMSE = 0.91049 Elapsed time : 0 sec
Training // Epoch 456 //  Total cost = 54417.10 Elapsed time : 1 sec
Testing // Epoch 456 //  Total cost = 214051.19  RMSE = 0.91331 Elapsed time : 0 sec
Training // Epoch 457 //  Total cost = 54441.84 Elapsed time : 1 sec
Testing // Epoch 457 //  Total cost = 212785.88  RMSE = 0.91018 Elapsed time : 0 sec
Training // Epoch 458 //  Total cost = 54653.03 Elapsed time : 1 sec
Testing // Epoch 458 //  Total cost = 212755.83  RMSE = 0.91009 Elapsed time : 0 sec
Training // Epoch 459 //  Total cost = 54596.75 Elapsed time : 1 sec
Testing // Epoch 459 //  Total cost = 213268.55  RMSE = 0.91145 Elapsed time : 0 sec
Training // Epoch 460 //  Total cost = 54689.51 Elapsed time : 1 sec
Testing // Epoch 460 //  Total cost = 213079.81  RMSE = 0.91101 Elapsed time : 0 sec
Training // Epoch 461 //  Total cost = 54563.66 Elapsed time : 1 sec
Testing // Epoch 461 //  Total cost = 212759.45  RMSE = 0.91024 Elapsed time

Training // Epoch 488 //  Total cost = 53402.87 Elapsed time : 1 sec
Testing // Epoch 488 //  Total cost = 211842.78  RMSE = 0.90826 Elapsed time : 0 sec
Training // Epoch 489 //  Total cost = 53415.55 Elapsed time : 1 sec
Testing // Epoch 489 //  Total cost = 212264.53  RMSE = 0.90945 Elapsed time : 0 sec
Training // Epoch 490 //  Total cost = 53393.55 Elapsed time : 1 sec
Testing // Epoch 490 //  Total cost = 211865.02  RMSE = 0.90830 Elapsed time : 0 sec
Training // Epoch 491 //  Total cost = 53453.55 Elapsed time : 1 sec
Testing // Epoch 491 //  Total cost = 211764.83  RMSE = 0.90806 Elapsed time : 0 sec
Training // Epoch 492 //  Total cost = 53415.23 Elapsed time : 1 sec
Testing // Epoch 492 //  Total cost = 212191.30  RMSE = 0.90922 Elapsed time : 0 sec
Training // Epoch 493 //  Total cost = 53343.61 Elapsed time : 1 sec
Testing // Epoch 493 //  Total cost = 212212.28  RMSE = 0.90930 Elapsed time : 0 sec
Training // Epoch 494 //  Total cost = 53396.50 Elapsed time : 1 sec
Testing